In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
positive_keyword_data = pd.read_csv('/content/drive/MyDrive/kt_aivle/big_project/data/used/df_positive_keyword.csv')
image_url_data = pd.read_excel('/content/drive/MyDrive/kt_aivle/big_project/data/used/data_final_07_10.xlsx')

In [ ]:
df_1 =positive_keyword_data[['콘텐츠명', '구명', '새주소[도로명 주소]', '지번주소', '좌표[X]', '좌표[Y]',
                             '전화번호', '대분류', '중분류', '실내_실외', 'naver_review_cnt', 'kakao_review_cnt',
                             'positive_keyword', 'positive_review', 'total_clean_review', 'ai_score', '유아용품', '키즈메뉴(아이가먹을수있는 메뉴)']].copy()

# ai_score를 선정한 리뷰 데이터 개수
def count_strings(lst):
    if isinstance(lst, str):
        # 문자열을 리스트로 변환
        lst = eval(lst)
    if isinstance(lst, list):
        return len(lst)
    else:
        return 0

df_1['review_cnt_for_ai_score'] = df_1['total_clean_review'].apply(count_strings)
df_1.drop(columns = ['total_clean_review'], inplace = True)

# 키워드 전처리
def preprocessed_positive_keyword(lst):
    lst = lst.strip('[]')
    lst = lst.replace(',', ', ')
    lst = lst.replace("'", "")
    lst = lst.replace('"', "")
    return lst
df_1['positive_keyword'] = df_1['positive_keyword'].apply(preprocessed_positive_keyword)

# positive_review 전처리
def preprocessed_positive_review(lst):
    lst = lst.strip('[]')
    return lst
df_1['positive_review'] = df_1['positive_review'].apply(preprocessed_positive_review)


df_2 = image_url_data[['영업시간', '주차가능여부', '무료구분', '대표이미지', '시설소개', '카테고리_키워드', '전체_키워드']].copy()

total_df = pd.concat([df_1, df_2], axis = 1)

total_df['영업시간'] = total_df['영업시간'].str.replace('\n', '\#')

In [ ]:
total_df[total_df['대표이미지'].isna()][['콘텐츠명', '대표이미지', 'ai_score']]

,콘텐츠명,대표이미지,ai_score
88,롯데리아 신촌로터리,NaN,0.0
173,엉터리서민갈비중앙대점,NaN,0.0
183,순두부와 청국장 그리고 찜,NaN,0.0
772,키즈팡,NaN,0.0


In [ ]:
total_df.drop(index = [88, 173, 183, 772], inplace = True)

In [ ]:
total_df[total_df['영업시간'].notnull()]['영업시간']

76     평일, 주말, 공휴일\#11:00~22:00
77                    연중무휴 24시간
78     평일, 주말, 공휴일\#11:30~21:30
79     평일, 주말, 공휴일\#11:30~22:30
80     평일, 주말, 공휴일\#08:00~22:00
                 ...           
883                         화~일
884                         월~토
885       화요일 ~ 일요일 10:00~17:30
886          화요일 ~ 일요일 (공휴일 제외)
887                         월-토
Name: 영업시간, Length: 596, dtype: object

In [ ]:
import re

# 숲체험, 및 유아숲 관련 키워드 정리

keyword_df = pd.DataFrame({
    "대분류" : ['먹기'] * 8 + ['배우기'] * 6 + ['놀기'] * 4,
    "중분류" : ['한식', '양식', '중식', '일식', '제과', '카페', '패스트푸드', '아시아푸드',
                '공연장/문화예술회관', '도서관', '미술관', '박물관', '기념관', '문화원',
                '숲체험', '공공형/서울형 키즈카페', '민간 키즈카페', '오락실']
    })

keywords_data = {
    ('먹기', '') : "음식점, 식사, 식당, 메뉴, 푸드, 맛있는, 맛,",
    ('먹기', '한식') : "한정식, 밥, 한식,",
    ('먹기', '양식') : "양식,",
    ('먹기', '중식') : "중국집, 중화요리, 자장면, 짬봉, 중식, 탕수육,",
    ('먹기', '일식') : "튀김, 일식, 경양식,",
    ('먹기', '제과') : "빵집, 베이커리, 제빵, 케이크, 도넛, 제과,",
    ('먹기', '카페') : "음료, 디저트, 에이드, 커피, 주스, 카페,",
    ('먹기', '패스트푸드') : "햄버거, 패스트푸드, 정커푸드,",
    ('먹기', '아시아푸드') : "아시아푸드, 향, 향신료, 베트남, 동남아, 아시아, 쌀국수, 태국,",

    ('배우기', '') : "문화시설, 공공시설, 공공기관, 창의력, 지식, 교양, 유익한,",
    ('배우기', '공연장/문화예술회관') : "공연장, 문화예술회관, 예술회관, 예술, 문화예술, 프로그램, 문화 프로그램, 관람, 클래식, 공연, 극장, 연극, 전시, 복합시설,",
    ('배우기', '도서관') : "도서관, 독서, 어린이자료실, 공공도서관, 한적한, 분위기 좋은, 주차장, 조용한, 책, 동화책, 영어원서, 서적, 그림책, 만화, 독서 프로그램, 유아용 책, 어린이 열람실, 유모차 출입, 독서 이벤트,",
    ('배우기', '미술관') : "미술관, 전시관, 갤러리, 전시, 미술, 작품, 작가, 그림, 굿즈,",
    ('배우기', '박물관') : "박물관, 역사, 전통적인, 과거, 시대, 기록, 전통, 테마, 문화유산, 현대사, 문명, 유적,",
    ('배우기', '기념관') : "기념관, 문화, 기념 행사, 전시, 역사, 전통적인, 과거, 시대, 기록, 전통,",
    ('배우기', '문화원') : "문화원, 문화센터, 프로그램, 배움, 강의, 강좌, 취미생활, 저렴함, 문화생활, 상담, 강사님, 행사, 체험, 체험활동, 수업, 수강, 축제, 교실, 다문화,",

    ('놀기', '') : "활동적인, 쉼터, 놀이, 게임, 놀이시설, 놀이터, 즐거운, 재밌는,",
    ('놀기', '숲체험') : "예약, 체험활동, 숲, 숲체험, 체험학습, 자연, 관찰, 깨끗한, 피크닉, 산책, 식물, 동물, 나무, 곤충, 꽃, 채집, 숲속놀이터,",
    ('놀기', '공공형/서울형 키즈카페') : "예약, 관리, 쾌적한, 넓은, 깨끗한, 안전한, 위생적인, 키즈카페, 공공형 키즈카페, 서울형 키즈카페, 강남형 키즈카페, 키즈카페, 휴식, 수유실,",
    ('놀기', '민간 키즈카페') : "키즈카페, 휴식, 수유실, 쾌적한, 넓은, 깨끗한, 안전한, 위생적인,",
    ('놀기', '오락실') : "게임, 좁은, 오락시설, 오락실, 인형뽑기, 인형, 자동차, 레이싱, 총,",
}

# 먼저 대분류 키워드를 추가
keyword_df['카테고리_키워드'] = keyword_df['대분류'].map(lambda x: keywords_data.get((x, ""), ""))
keyword_df['카테고리_키워드'] = keyword_df.apply(lambda row: row['카테고리_키워드'] + (keywords_data[(row['대분류'], row['중분류'])] if (row['대분류'], row['중분류']) in keywords_data else ""), axis=1)

# 기존 카테고리_키워드 컬럼 삭제 후, 키워드 변경사항 병합
total_df.drop(columns = ['카테고리_키워드'], inplace = True)
data = total_df.merge(keyword_df, on = ['대분류', '중분류'], how = 'left')


# 주차장이 있다면, 주차장이라는 키워드 추가. 없으면 검색되지 않는 의미로 추가 X
data.loc[data['주차가능여부'] == 'O', '카테고리_키워드'] += '주차장,'

# 이용료 또는 금액이 무료이면, 무료 키워드 추가, 유료이면 유료 키워드 추가, 없는 값은 불확실한 정보이므로 표기 X
data.loc[(data['무료구분'] == '무료') | (data['무료구분'] == 'Y'), '카테고리_키워드'] += '무료,'
data.loc[(data['무료구분'] == '유료') | (data['무료구분'] == 'N'), '카테고리_키워드'] += '유료,'

# 유아의자, 유아체어, 아기의자, 유아용의자, 아기의자,
data.loc[(data['유아용품'].str.contains('의자')) | (data['유아용품'].str.contains('의자')), '카테고리_키워드'] += '유아의자, 유아체어, 어린이의자, 어린이체어,'

def clean_kids_menu(text):
    text = re.sub(r'<.*?>', '', text)  # HTML 태그 제거
    text = re.sub(r'[\r\n]+', ', ', text)  # 개행문자 제거
    text = text.replace('/', ', ')
    text = text.replace('nan', '')
    return text

data['키즈메뉴(아이가먹을수있는 메뉴)'] = data['키즈메뉴(아이가먹을수있는 메뉴)'].astype(str).apply(clean_kids_menu)

data['전체_키워드'] = data['카테고리_키워드'] + "," +  data['키즈메뉴(아이가먹을수있는 메뉴)'].fillna('') + "," + data['positive_keyword']

data.drop(columns = ['키즈메뉴(아이가먹을수있는 메뉴)', '유아용품', '카테고리_키워드'], inplace = True)

In [ ]:
category_data = keyword_df[['대분류', '중분류', '카테고리_키워드']].copy()
category_data.to_excel('/content/drive/MyDrive/kt_aivle/big_project/data/preprocessed_data/category_data.xlsx', index=False)

In [ ]:
data.to_excel('/content/drive/MyDrive/kt_aivle/big_project/data/preprocessed_data/data_for_recommendation_07_15.xlsx', index=False)